In [1]:
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
import keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import pandas as pd
import numpy as np
from datetime import datetime
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "2"  

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7999)])
#   except RuntimeError as e:
#     print(e)
    
# tf.random.set_seed(1234)


2025-08-04 16:30:59.443519: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-04 16:31:00.244366: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Conventional methods

In [2]:
# Load Dataset
file_path = "/home1/nhuynh2023/datasets/PDF_HEA_Gibbs/HEA_Dataset_with_embeddings.csv"
dataset = pd.read_csv(file_path)

# Parse the embedding column to convert from string to a list
dataset["embedding"] = dataset["embedding"].apply(ast.literal_eval)

# Remove non-PDF features and irrelevant columns
pdf_columns = [col for col in dataset.columns if col.startswith("g(r)_") or col.startswith("r_")]
numerical_features = dataset.drop(columns=["ID","Gibbs", "prompt", "n_tokens", 'active_site_1', 'active_site_2', 'Fe', 'Co', 'Ni', 'Cu', 'Zn'])

# Filter to numerical types (PDF features)
numerical_features = numerical_features.select_dtypes(include=[np.number])

# Debug: Check remaining columns
print("Remaining columns after dropping irrelevant ones:")
print(numerical_features.columns.tolist())

# Standardize numerical features
scaler = StandardScaler()
pdf_features_scaled = scaler.fit_transform(numerical_features)

# Reduce dimensionality with PCA
m = 50  # Number of PCA components
pca = PCA(n_components=m)
pdf_pca = pca.fit_transform(pdf_features_scaled)

# Target variable
y = dataset["Gibbs"].values

# Debug: Output shapes
print("Shape of pdf_pca:", pdf_pca.shape)
print("Shape of y:", y.shape)

# Split into train and test sets
X_idx = np.arange(len(y))
X_train_idx, X_test_idx = train_test_split(X_idx, test_size=0.2, random_state=42)

Remaining columns after dropping irrelevant ones:
['r_0.2', 'g(r)_0.2', 'r_0.21', 'g(r)_0.21', 'r_0.22', 'g(r)_0.22', 'r_0.23', 'g(r)_0.23', 'r_0.24', 'g(r)_0.24', 'r_0.25', 'g(r)_0.25', 'r_0.26', 'g(r)_0.26', 'r_0.27', 'g(r)_0.27', 'r_0.28', 'g(r)_0.28', 'r_0.29', 'g(r)_0.29', 'r_0.3', 'g(r)_0.3', 'r_0.31', 'g(r)_0.31', 'r_0.32', 'g(r)_0.32', 'r_0.33', 'g(r)_0.33', 'r_0.34', 'g(r)_0.34', 'r_0.35', 'g(r)_0.35', 'r_0.36', 'g(r)_0.36', 'r_0.37', 'g(r)_0.37', 'r_0.38', 'g(r)_0.38', 'r_0.39', 'g(r)_0.39', 'r_0.4', 'g(r)_0.4', 'r_0.41', 'g(r)_0.41', 'r_0.42', 'g(r)_0.42', 'r_0.43', 'g(r)_0.43', 'r_0.44', 'g(r)_0.44', 'r_0.45', 'g(r)_0.45', 'r_0.46', 'g(r)_0.46', 'r_0.47', 'g(r)_0.47', 'r_0.48', 'g(r)_0.48', 'r_0.49', 'g(r)_0.49', 'r_0.5', 'g(r)_0.5', 'r_0.51', 'g(r)_0.51', 'r_0.52', 'g(r)_0.52', 'r_0.53', 'g(r)_0.53', 'r_0.54', 'g(r)_0.54', 'r_0.55', 'g(r)_0.55', 'r_0.56', 'g(r)_0.56', 'r_0.5700000000000001', 'g(r)_0.5700000000000001', 'r_0.58', 'g(r)_0.58', 'r_0.59', 'g(r)_0.59', 'r_0.6', 

In [15]:
# Prepare train and test data for conventional models
X_train = pdf_pca[X_train_idx]
X_test = pdf_pca[X_test_idx]
y_train = y[X_train_idx]
y_test = y[X_test_idx]

# Define conventional models and hyperparameter grids
models = {
    "RandomForest": {
        "model": RandomForestRegressor(random_state=42),
        "param_grid": {
            "n_estimators": [50, 100],
            "max_depth": [5, 10, None],
            "min_samples_leaf": [1, 5]
        }
    },
    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "param_grid": {
            "n_estimators": [50, 100],
            "learning_rate": [0.05, 0.1],
            "max_depth": [3, 5],
            "min_samples_leaf": [1, 5]
        }
    },
    "SVR": {
        "model": SVR(),
        "param_grid": {
            "C": [0.1, 1.0, 10.0],
            "epsilon": [0.01, 0.1],
            "kernel": ["rbf"]
        }
    },
    "LinearRegression": {
        "model": LinearRegression(),
        "param_grid": {}  # No hyperparameters to tune
    }
}

# Function to evaluate model performance
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} Performance:")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    return {"MAE": mae, "RMSE": rmse, "R²": r2}

# Train and evaluate each model
results = {}
for model_name, config in models.items():
    print(f"\nTraining {model_name}...")
    model = config["model"]
    param_grid = config["param_grid"]
    
    if param_grid:  # Perform grid search for models with hyperparameters
        grid_search = GridSearchCV(
            model, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1
        )
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    else:  # Fit directly for Linear Regression
        best_model = model
        best_model.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = best_model.predict(X_test)
    
    # Evaluate and store results
    results[model_name] = evaluate_model(y_test, y_pred, model_name)

# Save results to a DataFrame for comparison
results_df = pd.DataFrame(results).T
print("\nPerformance Comparison:")
print(results_df)

# Optionally, save results to a CSV file
results_df.to_csv("/home1/nhuynh2023/Projects/PDF_HEA_Gibbs/conventional_methods/conventional_models_performance.csv")


Training RandomForest...
Best parameters for RandomForest: {'max_depth': 5, 'min_samples_leaf': 5, 'n_estimators': 100}
RandomForest Performance:
MAE: 0.1714
RMSE: 0.2191
R²: 0.0963

Training GradientBoosting...
Best parameters for GradientBoosting: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 5, 'n_estimators': 50}
GradientBoosting Performance:
MAE: 0.1712
RMSE: 0.2177
R²: 0.1078

Training SVR...
Best parameters for SVR: {'C': 1.0, 'epsilon': 0.1, 'kernel': 'rbf'}
SVR Performance:
MAE: 0.1713
RMSE: 0.2200
R²: 0.0885

Training LinearRegression...
LinearRegression Performance:
MAE: 0.1674
RMSE: 0.2142
R²: 0.1357

Performance Comparison:
                       MAE      RMSE        R²
RandomForest      0.171405  0.219051  0.096320
GradientBoosting  0.171228  0.217659  0.107763
SVR               0.171316  0.219995  0.088512
LinearRegression  0.167448  0.214221  0.135727


# Methodologies

## Only PDF

In [3]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Load Dataset
file_path = "/home1/nhuynh2023/datasets/PDF_HEA_Gibbs/HEA_Dataset_with_embeddings.csv"
dataset = pd.read_csv(file_path)

# Parse the embedding column to convert from string to a list
dataset["embedding"] = dataset["embedding"].apply(ast.literal_eval)

# Remove PDF features (columns starting with "g(r)_" and "r_")
pdf_columns = [col for col in dataset.columns if col.startswith("g(r)_") or col.startswith("r_")]
numerical_features = dataset.drop(columns=["ID", "Gibbs", "prompt", "n_tokens", 'active_site_1', 'active_site_2', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'embedding'])


# Filter to numerical types (should be PDF features)
numerical_features = numerical_features.select_dtypes(include=[np.number])

# Debug: Check remaining columns
print("Remaining columns after dropping irrelevant ones:")
print(numerical_features.columns.tolist())


# Standardize numerical features
scaler = StandardScaler()
pdf_features_scaled = scaler.fit_transform(numerical_features)

# Reduce dimensionality with PCA
m = 50  # Number of PCA components, adjust as needed
pca = PCA(n_components=m)
pdf_pca = pca.fit_transform(pdf_features_scaled)

# Convert embeddings to array
embeddings_array = np.vstack(dataset["embedding"].values)

# Target variable
y = dataset["Gibbs"].values

# Debug: Output shapes
print("Shape of pdf_pca:", pdf_pca.shape)
print("Shape of embeddings:", embeddings_array.shape)
print("Shape of y:", y.shape)

# Split into train and test sets
X_idx = np.arange(len(y))
X_train_idx, X_test_idx = train_test_split(X_idx, test_size=0.2, random_state=42)

Remaining columns after dropping irrelevant ones:
['r_0.2', 'g(r)_0.2', 'r_0.21', 'g(r)_0.21', 'r_0.22', 'g(r)_0.22', 'r_0.23', 'g(r)_0.23', 'r_0.24', 'g(r)_0.24', 'r_0.25', 'g(r)_0.25', 'r_0.26', 'g(r)_0.26', 'r_0.27', 'g(r)_0.27', 'r_0.28', 'g(r)_0.28', 'r_0.29', 'g(r)_0.29', 'r_0.3', 'g(r)_0.3', 'r_0.31', 'g(r)_0.31', 'r_0.32', 'g(r)_0.32', 'r_0.33', 'g(r)_0.33', 'r_0.34', 'g(r)_0.34', 'r_0.35', 'g(r)_0.35', 'r_0.36', 'g(r)_0.36', 'r_0.37', 'g(r)_0.37', 'r_0.38', 'g(r)_0.38', 'r_0.39', 'g(r)_0.39', 'r_0.4', 'g(r)_0.4', 'r_0.41', 'g(r)_0.41', 'r_0.42', 'g(r)_0.42', 'r_0.43', 'g(r)_0.43', 'r_0.44', 'g(r)_0.44', 'r_0.45', 'g(r)_0.45', 'r_0.46', 'g(r)_0.46', 'r_0.47', 'g(r)_0.47', 'r_0.48', 'g(r)_0.48', 'r_0.49', 'g(r)_0.49', 'r_0.5', 'g(r)_0.5', 'r_0.51', 'g(r)_0.51', 'r_0.52', 'g(r)_0.52', 'r_0.53', 'g(r)_0.53', 'r_0.54', 'g(r)_0.54', 'r_0.55', 'g(r)_0.55', 'r_0.56', 'g(r)_0.56', 'r_0.5700000000000001', 'g(r)_0.5700000000000001', 'r_0.58', 'g(r)_0.58', 'r_0.59', 'g(r)_0.59', 'r_0.6', 

In [19]:
# Custom Dataset
class HEAGibbsDataset(Dataset):
    def __init__(self, indices):
        self.pdf_pca = torch.tensor(pdf_pca[indices], dtype=torch.float32)
        self.embeddings = torch.tensor(embeddings_array[indices], dtype=torch.float32)
        self.y = torch.tensor(y[indices], dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.pdf_pca[idx], self.embeddings[idx], self.y[idx]

# Create datasets and loaders
train_dataset = HEAGibbsDataset(X_train_idx)
test_dataset = HEAGibbsDataset(X_test_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define Transformer Model
class TransformerRegressor(nn.Module):
    def __init__(self, m, d, d_model=64, nhead=8, num_layers=2):
        super().__init__()
        # Embed PCA components (scalars) to d_model
        self.pca_embed = nn.Linear(1, d_model)
        # Project embedding vector to d_model
        self.embed_proj = nn.Linear(d, d_model)
        # Positional encoding
        self.pos_encoder = nn.Parameter(torch.zeros(1, m + 1, d_model))
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # Regression head
        self.regression_head = nn.Linear(d_model, 1)

    def forward(self, pdf_pca, embeddings):
        # pdf_pca: (batch_size, m)
        # embeddings: (batch_size, d)
        batch_size = pdf_pca.size(0)

        # Embed PCA components
        pca_tokens = self.pca_embed(pdf_pca.unsqueeze(-1))  # (batch_size, m, d_model)
        # Project embeddings
        embed_token = self.embed_proj(embeddings).unsqueeze(1)  # (batch_size, 1, d_model)
        # Concatenate: [embedding, PCA tokens]
        tokens = torch.cat([embed_token, pca_tokens], dim=1)  # (batch_size, m+1, d_model)
        # Add positional encodings
        tokens = tokens + self.pos_encoder
        # Pass through transformer
        output = self.transformer_encoder(tokens)  # (batch_size, m+1, d_model)
        # Use first token (embedding) for prediction
        cls_output = output[:, 0, :]  # (batch_size, d_model)
        # Predict Gibbs
        y_pred = self.regression_head(cls_output).squeeze(-1)  # (batch_size,)
        return y_pred

In [21]:
# Instantiate model
d = embeddings_array.shape[1]  # Embedding dimension
model = TransformerRegressor(m=m, d=d, d_model=64, nhead=8, num_layers=2)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 200
best_test_loss = float('inf')  # Initialize with infinity
best_model_state = None        # To store the best model's state_dict

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for pdf_pca_batch, embeddings_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(pdf_pca_batch, embeddings_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Evaluate on test set
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for pdf_pca_batch, embeddings_batch, y_batch in test_loader:
            y_pred = model(pdf_pca_batch, embeddings_batch)
            test_loss += criterion(y_pred, y_batch).item()
        test_loss /= len(test_loader)
        
    
    # Update best model if test loss improves
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_model_state = model.state_dict()  # Store the state_dict in memory
        print(f"Epoch {epoch+1}: New best test loss found: {best_test_loss:.4f}")

    # Print progress
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
    
# After training, save the best model
if best_model_state is not None:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'/home1/nhuynh2023/Projects/PDF_HEA_Gibbs/models/Only_PDF/model_{timestamp}_loss_{best_test_loss:.4f}.pth'
    torch.save(best_model_state, filename)
    print(f"Training complete. Best model saved to {filename} with Test Loss: {best_test_loss:.4f}")
else:
    print("Training complete. No model saved (no improvement found).")

Epoch 1: New best test loss found: 0.0578
Epoch 1, Train Loss: 0.2663, Test Loss: 0.0578
Epoch 2: New best test loss found: 0.0565
Epoch 2, Train Loss: 0.0618, Test Loss: 0.0565
Epoch 3: New best test loss found: 0.0551
Epoch 3, Train Loss: 0.0650, Test Loss: 0.0551
Epoch 4, Train Loss: 0.0595, Test Loss: 0.0585
Epoch 5: New best test loss found: 0.0538
Epoch 5, Train Loss: 0.0580, Test Loss: 0.0538
Epoch 6, Train Loss: 0.0577, Test Loss: 0.0622
Epoch 7, Train Loss: 0.0594, Test Loss: 0.0544
Epoch 8, Train Loss: 0.0575, Test Loss: 0.0559
Epoch 9: New best test loss found: 0.0533
Epoch 9, Train Loss: 0.0563, Test Loss: 0.0533
Epoch 10, Train Loss: 0.0576, Test Loss: 0.0553
Epoch 11: New best test loss found: 0.0507
Epoch 11, Train Loss: 0.0557, Test Loss: 0.0507
Epoch 12: New best test loss found: 0.0190
Epoch 12, Train Loss: 0.0431, Test Loss: 0.0190
Epoch 13, Train Loss: 0.0273, Test Loss: 0.0214
Epoch 14: New best test loss found: 0.0166
Epoch 14, Train Loss: 0.0210, Test Loss: 0.016

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model.load_state_dict(torch.load("/home1/nhuynh2023/Projects/PDF_HEA_Gibbs/models/Only_PDF/model_20250226_122617_loss_0.0020.pth"))
model.eval()

# Function to collect predictions and actual values from the model
def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []  # List to store predicted values
    actuals = []      # List to store actual values
    with torch.no_grad():  # Disable gradient computation for efficiency
        for pdf_pca_batch, embeddings_batch, y_batch in data_loader:
            # Forward pass through the model
            y_pred = model(pdf_pca_batch, embeddings_batch)
            # Move predictions and actuals to CPU and convert to NumPy
            predictions.append(y_pred.cpu().numpy())
            actuals.append(y_batch.cpu().numpy())
    # Combine all batches into single arrays
    predictions = np.concatenate(predictions)
    actuals = np.concatenate(actuals)
    return predictions, actuals

# Get predictions and actual values for training and test sets
train_predictions, train_actuals = evaluate_model(model, train_loader)
test_predictions, test_actuals = evaluate_model(model, test_loader)

# Function to calculate MAE, RMSE, and R²
def calculate_metrics(predictions, actuals):
    mae = mean_absolute_error(actuals, predictions)  # Mean Absolute Error
    mse = mean_squared_error(actuals, predictions)   # Mean Squared Error
    rmse = np.sqrt(mse)                              # Root Mean Squared Error
    r2 = r2_score(actuals, predictions)              # R-squared
    return mae, rmse, r2

# Calculate metrics for training and test sets
train_mae, train_rmse, train_r2 = calculate_metrics(train_predictions, train_actuals)
test_mae, test_rmse, test_r2 = calculate_metrics(test_predictions, test_actuals)

# Print the results
print(f"Training MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}")
print(f"Test MAE: {test_mae:.4f}, RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}")

Training MAE: 0.0291, RMSE: 0.0387, R²: 0.9730
Test MAE: 0.0346, RMSE: 0.0464, R²: 0.9594


## Elements and active sites

In [4]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Load Dataset
file_path = "/home1/nhuynh2023/datasets/PDF_HEA_Gibbs/HEA_Dataset_with_embeddings.csv"
dataset = pd.read_csv(file_path)

# Define the elements
elements = ['Fe', 'Co', 'Ni', 'Cu', 'Zn']

# Standardize the composition columns
scaler = StandardScaler()
dataset[elements] = scaler.fit_transform(dataset[elements])

# Function to create input sequence for each sample
def create_input_sequence(row, elements):
    # Extract compositions
    compositions = [row[element] for element in elements]
    # Get active sites (assumed to be strings like 'Fe', 'Co', etc.)
    active_site_1 = row['active_site_1']
    active_site_2 = row['active_site_2']
    # Create binary indicators for active sites
    is_active_site_1 = [1 if element == active_site_1 else 0 for element in elements]
    is_active_site_2 = [1 if element == active_site_2 else 0 for element in elements]
    # Define [CLS] token
    cls_token = [0, 0, 0]  # Placeholder for [CLS]
    # Create element tokens: [composition, is_active_site_1, is_active_site_2]
    element_tokens = [
        [comp, is_as1, is_as2]
        for comp, is_as1, is_as2 in zip(compositions, is_active_site_1, is_active_site_2)
    ]
    # Combine into sequence: [CLS] + element tokens
    sequence = [cls_token] + element_tokens
    return sequence

# Generate input sequences for all samples
input_sequences = np.array(
    dataset.apply(lambda row: create_input_sequence(row, elements), axis=1).tolist()
)

# Target variable
y = dataset["Gibbs"].values

# Debug: Output shapes
print("Shape of input_sequences:", input_sequences.shape)  # Expected: (n_samples, 6, 3)
print("Shape of y:", y.shape)

Shape of input_sequences: (1268, 6, 3)
Shape of y: (1268,)


In [5]:
# Split into train and test sets
X_idx = np.arange(len(y))
X_train_idx, X_test_idx = train_test_split(X_idx, test_size=0.2, random_state=42)

# Define Custom Dataset
class HEAGibbsDataset(Dataset):
    def __init__(self, indices, input_sequences, y):
        self.input_sequences = torch.tensor(input_sequences[indices], dtype=torch.float32)
        self.y = torch.tensor(y[indices], dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.input_sequences[idx], self.y[idx]

# Create Datasets
train_dataset = HEAGibbsDataset(X_train_idx, input_sequences, y)
test_dataset = HEAGibbsDataset(X_test_idx, input_sequences, y)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# Define Transformer Model
class TransformerRegressor(nn.Module):
    def __init__(self, d_model=64, nhead=8, num_layers=2):
        super().__init__()
        # Embed 3D input features to d_model
        self.input_embed = nn.Linear(3, d_model)
        # Positional encoding for sequence length 6
        self.pos_encoder = nn.Parameter(torch.zeros(1, 6, d_model))
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # Regression head
        self.regression_head = nn.Linear(d_model, 1)

    def forward(self, input_sequences):
        # input_sequences: (batch_size, 6, 3)
        tokens = self.input_embed(input_sequences)  # (batch_size, 6, d_model)
        tokens = tokens + self.pos_encoder  # Add positional encodings
        output = self.transformer_encoder(tokens)  # (batch_size, 6, d_model)
        cls_output = output[:, 0, :]  # Use [CLS] token output: (batch_size, d_model)
        y_pred = self.regression_head(cls_output).squeeze(-1)  # (batch_size,)
        return y_pred

# Instantiate Model
model = TransformerRegressor(d_model=64, nhead=8, num_layers=2)

# (Optional) Define optimizer and loss function for training
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [ ]:
# Training Loop with Saving Only the Best Model at the End
num_epochs = 200
best_test_loss = float('inf')  # Initialize with infinity
best_model_state = None        # To store the best model's state_dict

for epoch in range(num_epochs):
    # Training Phase
    model.train()
    train_loss = 0
    for input_sequences_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(input_sequences_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Evaluation Phase
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for input_sequences_batch, y_batch in test_loader:
            y_pred = model(input_sequences_batch)
            loss = criterion(y_pred, y_batch)
            test_loss += loss.item()
        test_loss /= len(test_loader)

    # Update best model if test loss improves
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_model_state = model.state_dict()  # Store the state_dict in memory
        print(f"Epoch {epoch+1}: New best test loss found: {best_test_loss:.4f}")

    # Print progress
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    
# After training, save the best model
if best_model_state is not None:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'/home1/nhuynh2023/Projects/PDF_HEA_Gibbs/models/Ele_As/model_{timestamp}_loss_{best_test_loss:.4f}.pth'
    torch.save(best_model_state, filename)
    print(f"Training complete. Best model saved to {filename} with Test Loss: {best_test_loss:.4f}")
else:
    print("Training complete. No model saved (no improvement found).")

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model.load_state_dict(torch.load("/home1/nhuynh2023/Projects/PDF_HEA_Gibbs/models/Ele_As/model_20250226_121417_loss_0.0072.pth"))
model.eval()

# Function to collect predictions and actual values from the model
def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []  # List to store predicted values
    actuals = []      # List to store actual values
    with torch.no_grad():  # Disable gradient computation for efficiency
        for input_sequences_batch, y_batch in data_loader:
            # Forward pass through the model
            y_pred = model(input_sequences_batch)
            # Move predictions and actuals to CPU and convert to NumPy
            predictions.append(y_pred.cpu().numpy())
            actuals.append(y_batch.cpu().numpy())
    # Combine all batches into single arrays
    predictions = np.concatenate(predictions)
    actuals = np.concatenate(actuals)
    return predictions, actuals

# Get predictions and actual values for training and test sets
train_predictions, train_actuals = evaluate_model(model, train_loader)
test_predictions, test_actuals = evaluate_model(model, test_loader)

# Function to calculate MAE, RMSE, and R²
def calculate_metrics(predictions, actuals):
    mae = mean_absolute_error(actuals, predictions)  # Mean Absolute Error
    mse = mean_squared_error(actuals, predictions)   # Mean Squared Error
    rmse = np.sqrt(mse)                              # Root Mean Squared Error
    r2 = r2_score(actuals, predictions)              # R-squared
    return mae, rmse, r2

# Calculate metrics for training and test sets
train_mae, train_rmse, train_r2 = calculate_metrics(train_predictions, train_actuals)
test_mae, test_rmse, test_r2 = calculate_metrics(test_predictions, test_actuals)

# Print the results
print(f"Training MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}")
print(f"Test MAE: {test_mae:.4f}, RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}")

Training MAE: 0.0661, RMSE: 0.0836, R²: 0.8740
Test MAE: 0.0766, RMSE: 0.0948, R²: 0.8308


## PDF and LLM embeddings

In [4]:
# Load Dataset
file_path = "/home1/nhuynh2023/datasets/PDF_HEA_Gibbs/HEA_Dataset_with_embeddings.csv"
dataset = pd.read_csv(file_path)

# Parse the embedding column to convert from string to a list
dataset["embedding"] = dataset["embedding"].apply(ast.literal_eval)

# Remove non-PDF features
pdf_columns = [col for col in dataset.columns if col.startswith("g(r)_") or col.startswith("r_")]
numerical_features = dataset.drop(columns=["ID", "Gibbs", "prompt", "n_tokens", 'active_site_1', 'active_site_2', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'embedding'])

# Filter to numerical types (should be PDF features)
numerical_features = numerical_features.select_dtypes(include=[np.number])

# Debug: Check remaining columns
print("Remaining columns after dropping irrelevant ones:")
print(numerical_features.columns.tolist())

# Standardize numerical features
scaler = StandardScaler()
pdf_features_scaled = scaler.fit_transform(numerical_features)

# Reduce dimensionality with PCA
m = 50  # Number of PCA components
pca = PCA(n_components=m)
pdf_pca = pca.fit_transform(pdf_features_scaled)

# Convert embeddings to array
embeddings_array = np.vstack(dataset["embedding"].values)

# Target variable
y = dataset["Gibbs"].values

# Debug: Output shapes
print("Shape of pdf_pca:", pdf_pca.shape)
print("Shape of embeddings:", embeddings_array.shape)
print("Shape of y:", y.shape)

Remaining columns after dropping irrelevant ones:
['r_0.2', 'g(r)_0.2', 'r_0.21', 'g(r)_0.21', 'r_0.22', 'g(r)_0.22', 'r_0.23', 'g(r)_0.23', 'r_0.24', 'g(r)_0.24', 'r_0.25', 'g(r)_0.25', 'r_0.26', 'g(r)_0.26', 'r_0.27', 'g(r)_0.27', 'r_0.28', 'g(r)_0.28', 'r_0.29', 'g(r)_0.29', 'r_0.3', 'g(r)_0.3', 'r_0.31', 'g(r)_0.31', 'r_0.32', 'g(r)_0.32', 'r_0.33', 'g(r)_0.33', 'r_0.34', 'g(r)_0.34', 'r_0.35', 'g(r)_0.35', 'r_0.36', 'g(r)_0.36', 'r_0.37', 'g(r)_0.37', 'r_0.38', 'g(r)_0.38', 'r_0.39', 'g(r)_0.39', 'r_0.4', 'g(r)_0.4', 'r_0.41', 'g(r)_0.41', 'r_0.42', 'g(r)_0.42', 'r_0.43', 'g(r)_0.43', 'r_0.44', 'g(r)_0.44', 'r_0.45', 'g(r)_0.45', 'r_0.46', 'g(r)_0.46', 'r_0.47', 'g(r)_0.47', 'r_0.48', 'g(r)_0.48', 'r_0.49', 'g(r)_0.49', 'r_0.5', 'g(r)_0.5', 'r_0.51', 'g(r)_0.51', 'r_0.52', 'g(r)_0.52', 'r_0.53', 'g(r)_0.53', 'r_0.54', 'g(r)_0.54', 'r_0.55', 'g(r)_0.55', 'r_0.56', 'g(r)_0.56', 'r_0.5700000000000001', 'g(r)_0.5700000000000001', 'r_0.58', 'g(r)_0.58', 'r_0.59', 'g(r)_0.59', 'r_0.6', 

In [38]:
# Split into train and test sets
X_idx = np.arange(len(y))
X_train_idx, X_test_idx = train_test_split(X_idx, test_size=0.2, random_state=42)

# Custom Dataset
class HEAGibbsDataset(Dataset):
    def __init__(self, indices):
        self.pdf_pca = torch.tensor(pdf_pca[indices], dtype=torch.float32)
        self.embeddings = torch.tensor(embeddings_array[indices], dtype=torch.float32)
        self.y = torch.tensor(y[indices], dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.pdf_pca[idx], self.embeddings[idx], self.y[idx]

# Create datasets and loaders
train_dataset = HEAGibbsDataset(X_train_idx)
test_dataset = HEAGibbsDataset(X_test_idx)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [39]:
# Define Transformer Model
class TransformerRegressor(nn.Module):
    def __init__(self, m, d, d_model=64, nhead=8, num_layers=2):
        super().__init__()
        # Embed PCA components (scalars) to d_model
        self.pca_embed = nn.Linear(1, d_model)
        # Project embedding vector to d_model
        self.embed_proj = nn.Linear(d, d_model)
        # Positional encoding
        self.pos_encoder = nn.Parameter(torch.zeros(1, m + 1, d_model))
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # Regression head
        self.regression_head = nn.Linear(d_model, 1)

    def forward(self, pdf_pca, embeddings):
        batch_size = pdf_pca.size(0)
        # Embed PCA components
        pca_tokens = self.pca_embed(pdf_pca.unsqueeze(-1))  # (batch_size, m, d_model)
        # Project embeddings
        embed_token = self.embed_proj(embeddings).unsqueeze(1)  # (batch_size, 1, d_model)
        # Concatenate: [GPT embedding, PCA tokens]
        tokens = torch.cat([embed_token, pca_tokens], dim=1)  # (batch_size, m+1, d_model)
        # Add positional encodings
        tokens = tokens + self.pos_encoder
        # Pass through transformer
        output = self.transformer_encoder(tokens)  # (batch_size, m+1, d_model)
        # Use first token (GPT embedding) for prediction
        cls_output = output[:, 0, :]  # (batch_size, d_model)
        y_pred = self.regression_head(cls_output).squeeze(-1)  # (batch_size,)
        return y_pred

# Instantiate model
d = embeddings_array.shape[1]  # Embedding dimension
model = TransformerRegressor(m=m, d=d, d_model=64, nhead=8, num_layers=2)

# Optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()



In [42]:
# Training Loop with Saving the Best Model
num_epochs = 200
best_test_loss = float('inf')
best_model_state = None

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0
    for pdf_pca_batch, embeddings_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(pdf_pca_batch, embeddings_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Evaluation
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for pdf_pca_batch, embeddings_batch, y_batch in test_loader:
            y_pred = model(pdf_pca_batch, embeddings_batch)
            loss = criterion(y_pred, y_batch)
            test_loss += loss.item()
        test_loss /= len(test_loader)

    # Save best model
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        best_model_state = model.state_dict()
        print(f"Epoch {epoch+1}: New best test loss: {best_test_loss:.4f}")

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

# Save the best model
if best_model_state is not None:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'/home1/nhuynh2023/Projects/PDF_HEA_Gibbs/models/PDF_GPT/model_{timestamp}_loss_{best_test_loss:.4f}.pth'
    torch.save(best_model_state, filename)
    print(f"Best model saved to {filename} with Test Loss: {best_test_loss:.4f}")
else:
    print("No model saved (no improvement found).")

Epoch 1: New best test loss: 0.0022
Epoch 1, Train Loss: 0.0033, Test Loss: 0.0022
Epoch 2, Train Loss: 0.0029, Test Loss: 0.0027
Epoch 3, Train Loss: 0.0030, Test Loss: 0.0041
Epoch 4: New best test loss: 0.0019
Epoch 4, Train Loss: 0.0031, Test Loss: 0.0019
Epoch 5, Train Loss: 0.0026, Test Loss: 0.0024
Epoch 6, Train Loss: 0.0035, Test Loss: 0.0024
Epoch 7, Train Loss: 0.0033, Test Loss: 0.0029
Epoch 8, Train Loss: 0.0032, Test Loss: 0.0030
Epoch 9, Train Loss: 0.0036, Test Loss: 0.0036
Epoch 10, Train Loss: 0.0039, Test Loss: 0.0029
Epoch 11, Train Loss: 0.0035, Test Loss: 0.0030
Epoch 12, Train Loss: 0.0035, Test Loss: 0.0022
Epoch 13, Train Loss: 0.0025, Test Loss: 0.0023
Epoch 14, Train Loss: 0.0030, Test Loss: 0.0026
Epoch 15, Train Loss: 0.0033, Test Loss: 0.0023
Epoch 16, Train Loss: 0.0026, Test Loss: 0.0025
Epoch 17, Train Loss: 0.0023, Test Loss: 0.0028
Epoch 18, Train Loss: 0.0028, Test Loss: 0.0019
Epoch 19, Train Loss: 0.0027, Test Loss: 0.0022
Epoch 20, Train Loss: 0.0

In [43]:

model.load_state_dict(torch.load("/home1/nhuynh2023/Projects/PDF_HEA_Gibbs/models/PDF_GPT/model_20250226_141802_loss_0.0014.pth"))
model.eval()

# Function to collect predictions and actual values from the model
def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []  # List to store predicted values
    actuals = []      # List to store actual values
    with torch.no_grad():  # Disable gradient computation for efficiency
        for pdf_pca_batch, gpt_embeddings_batch, y_batch in data_loader:
            # Forward pass through the model
            y_pred = model(pdf_pca_batch, gpt_embeddings_batch)
            # Move predictions and actuals to CPU and convert to NumPy
            predictions.append(y_pred.cpu().numpy())
            actuals.append(y_batch.cpu().numpy())
    # Combine all batches into single arrays
    predictions = np.concatenate(predictions)
    actuals = np.concatenate(actuals)
    return predictions, actuals

# Get predictions and actual values for training and test sets
train_predictions, train_actuals = evaluate_model(model, train_loader)
test_predictions, test_actuals = evaluate_model(model, test_loader)

# Function to calculate MAE, RMSE, and R²
def calculate_metrics(predictions, actuals):
    mae = mean_absolute_error(actuals, predictions)  # Mean Absolute Error
    mse = mean_squared_error(actuals, predictions)   # Mean Squared Error
    rmse = np.sqrt(mse)                              # Root Mean Squared Error
    r2 = r2_score(actuals, predictions)              # R-squared
    return mae, rmse, r2

# Calculate metrics for training and test sets
train_mae, train_rmse, train_r2 = calculate_metrics(train_predictions, train_actuals)
test_mae, test_rmse, test_r2 = calculate_metrics(test_predictions, test_actuals)

# Print the results
print(f"Training MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}")
print(f"Test MAE: {test_mae:.4f}, RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}")

Training MAE: 0.0273, RMSE: 0.0343, R²: 0.9788
Test MAE: 0.0332, RMSE: 0.0440, R²: 0.9635
